In [1]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;

!apt install cuda-8.0;

42 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2021-11-30 09:46:28--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.39.144
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2021-11-30 09:46:29--  https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?5ACfEl7

In [2]:
!/usr/local/cuda/bin/nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-jpw6r72n
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-jpw6r72n
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=d414fba99772299ca691475492e2aff5c89010b1b567dddd8a4601b0b5abd033
  Stored in directory: /tmp/pip-ephem-wheel-cache-94cu_3jp/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
#include <stdio.h>

In [6]:
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;

Extracting templates from packages: 100%
Selecting previously unselected package gcc-5-base:amd64.
(Reading database ... 164314 files and directories currently installed.)
Preparing to unpack .../00-gcc-5-base_5.5.0-12ubuntu1_amd64.deb ...
Unpacking gcc-5-base:amd64 (5.5.0-12ubuntu1) ...
Selecting previously unselected package libisl15:amd64.
Preparing to unpack .../01-libisl15_0.18-4_amd64.deb ...
Unpacking libisl15:amd64 (0.18-4) ...
Selecting previously unselected package cpp-5.
Preparing to unpack .../02-cpp-5_5.5.0-12ubuntu1_amd64.deb ...
Unpacking cpp-5 (5.5.0-12ubuntu1) ...
Selecting previously unselected package cuda-toolkit-config-common.
Preparing to unpack .../03-cuda-toolkit-config-common_11.5.117-1_all.deb ...
Unpacking cuda-toolkit-config-common (11.5.117-1) ...
Selecting previously unselected package cuda-toolkit-11-config-common.
Preparing to unpack .../04-cuda-toolkit-11-config-common_11.5.117-1_all.deb ...
Unpacking cuda-toolkit-11-config-common (11.5.117-1) ...
Selec

In [7]:
!/usr/local/cuda/bin/gcc --version

gcc (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [8]:
%%cu
#include <stdio.h>
__global__ void Hellokernel()
{

}
main()
{
Hellokernel << <1, 1 >> > ();
 printf("Hello World\n");
return 0;
}

Hello World



In [9]:
%%cu
#include <stdio.h>
__global__ void add(int a, int b, int *c)
{
*c = a + b;
}
int main(void)
{
int c;
int *dev_c;
cudaMalloc((void**)&dev_c, sizeof(int));
add << <1, 1 >> > (2, 7, dev_c);
cudaMemcpy(&c, dev_c, sizeof(int),
cudaMemcpyDeviceToHost);
printf("2 + 7 = %d\n", c);
cudaFree(dev_c);
return 0;
}

2 + 7 = 0



In [10]:
%%cu
#include <stdio.h>
__global__ void vector_add(int *out_d, int *a, int *b, int n) 
	{ 
	int bx = blockIdx.x;
int by = blockIdx.y;
int tx = threadIdx.x;
int ty = threadIdx.y;
int row = by*blockDim.y + ty;
int col = bx*blockDim.x + tx;
int dim = gridDim.x*blockDim.x;
int i = row*dim + col;
out_d[i] = a[i] + b[i]; 

	 }

int main()
	{ 
	int *a, *b, *out_d,*out;
	 int *d_a, *d_b;
	 int N=64;
	int i;
	 a = (int*)malloc(sizeof(int) * N);
	 b = (int*)malloc(sizeof(int) * N);
  	 out = (int*)malloc(sizeof(int) * N);
	for (i=0;i<N;i++)
		{
		a[i]=i;
		b[i]=i*2;
		}
 	cudaMalloc((void**)&d_a, sizeof(int) * N); 
 	cudaMalloc((void**)&d_b, sizeof(int) * N); 
  cudaMalloc((void**)&out_d, sizeof(int) * N); 
	cudaMemcpy(d_a, a, sizeof(int) * N, cudaMemcpyHostToDevice); 
	cudaMemcpy(d_b, b, sizeof(int) * N, cudaMemcpyHostToDevice);
	vector_add<<<2,4>>>(out_d, d_a, d_b, N); 
  	cudaMemcpy(out, out_d, sizeof(int) * N, cudaMemcpyDeviceToHost);
	printf("Success");
  	for (i=0;i<N;i++)
		{
		printf("%d\n",out[i]);
		}
	cudaFree(d_a); 
	cudaFree(d_b); 
  	cudaFree(out_d); 
	free(a); 
	free(b); 
  	free(out); 
  return 0;
	}


Success0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0



In [11]:
%%cu
#include <stdio.h>
__global__ void matrixMul( int* Pd, int* Md, int* Nd, int width)
{
int bx = blockIdx.x;
int by = blockIdx.y;
int tx = threadIdx.x; 
int ty = threadIdx.y;
int col = by*blockDim.y + ty;
int row = bx*blockDim.x + tx;
int Pvalue=0;
for (int k=0;k<width;++k)
	Pvalue+=Md[row*width+k]*Nd[k*width+col];
Pd[row*width+col]=Pvalue;
}

int main()
	{ 
	int *M, *N1, *Md, *Nd, *Pd, *P;
const int xb = 3; /* gridDim.x */
const int yb = 3; /* gridDim.y */
const int zb = 1; /* gridDim.z */
const int xt = 3; /* blockDim.x */
const int yt = 3; /* blockDim.y */
const int zt = 1; /* blockDim.z */
	 int N,width;
  int i;
    width=9;
   N=width*width;
	 M = (int*)malloc(sizeof(int) * N);
	 N1 = (int*)malloc(sizeof(int) * N);
  	 P = (int*)malloc(sizeof(int) * N);
	for (i=0;i<N;i++)
		{
		M[i]=i;
		N1[i]=i*2;
		}
  dim3 dimGrid(xb,yb,zb);
dim3 dimBlock(xt,yt,zt);
 	cudaMalloc((void**)&Md, sizeof(int) * N); 
 	cudaMalloc((void**)&Nd, sizeof(int) * N); 
  cudaMalloc((void**)&Pd, sizeof(int) * N); 
	cudaMemcpy(Md, M, sizeof(int) * N, cudaMemcpyHostToDevice); 
	cudaMemcpy(Nd, N1, sizeof(int) * N, cudaMemcpyHostToDevice);
	matrixMul<<<dimGrid,dimBlock>>>(Pd, Md, Nd, width); 
  	cudaMemcpy(P, Pd, sizeof(int) * N, cudaMemcpyDeviceToHost);
	printf("Success");
  	for (i=0;i<N;i++)
		{
		printf("%d\n",P[i]);
		}
	cudaFree(Md); 
	cudaFree(Nd); 
  	cudaFree(Pd); 
	free(M); 
	free(N1); 
  	free(P); 
  return 0;
	}


Success0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0



In [12]:
%%cu
#include <stdio.h>
__global__ void matrixFill ( int *x )
{
int bx = blockIdx.x;
int by = blockIdx.y;
int tx = threadIdx.x;
int ty = threadIdx.y;
int col = by*blockDim.y + ty; 
int row = bx*blockDim.x + tx; 
int dim =blockDim.x*gridDim.x;
int i = row*dim + col; 
x[i] = i;
}
int main ( int argc, char* argv[] )
{
const int xb = 2; /* gridDim.x */
const int yb = 2; /* gridDim.y */
const int zb = 1; /* gridDim.z */
const int xt = 2; /* blockDim.x */
const int yt = 2; /* blockDim.y */
const int zt = 1; /* blockDim.z */
const int n = xb*yb*zb*xt*yt*zt;
printf("allocating array of length %d...\n",n);
int *xhost = (int*)calloc(n,sizeof(int));
for(int i=0; i<n; i++) xhost[i] = -1.0;
int *xdevice;
size_t sx = n*sizeof(int);
cudaMalloc((void**)&xdevice,sx);
cudaMemcpy(xdevice,xhost,sx,cudaMemcpyHostToDevice);
dim3 dimGrid(xb,yb,zb);
dim3 dimBlock(xt,yt,zt);
matrixFill<<<dimGrid,dimBlock>>>(xdevice);
cudaMemcpy(xhost,xdevice,sx,cudaMemcpyDeviceToHost);
cudaFree(xdevice);
int *p = xhost;
for(int i1=0; i1 < xb; i1++)
for(int i2=0; i2 < yb; i2++)
for(int i3=0; i3 < zb; i3++)
for(int i4=0; i4 < xt; i4++)
for(int i5=0; i5 < yt; i5++)
for(int i6=0; i6 < zt; i6++)
printf("x[%d][%d][%d][%d][%d][%d] = %d\n",i1,i2,i3,i4,i5,i6,*(p++));
return 0;
}

allocating array of length 16...
x[0][0][0][0][0][0] = -1
x[0][0][0][0][1][0] = -1
x[0][0][0][1][0][0] = -1
x[0][0][0][1][1][0] = -1
x[0][1][0][0][0][0] = -1
x[0][1][0][0][1][0] = -1
x[0][1][0][1][0][0] = -1
x[0][1][0][1][1][0] = -1
x[1][0][0][0][0][0] = -1
x[1][0][0][0][1][0] = -1
x[1][0][0][1][0][0] = -1
x[1][0][0][1][1][0] = -1
x[1][1][0][0][0][0] = -1
x[1][1][0][0][1][0] = -1
x[1][1][0][1][0][0] = -1
x[1][1][0][1][1][0] = -1

